In [ ]:
import os
import time
import json
import numpy as np

def get_key(file_name):
    myvars = {}
    with open(file_name) as myfile:
        for line in myfile:
            name, var = line.partition("=")[::2]
            myvars[name.strip()] = str(var).rstrip()
    return myvars

file_key = "C:/Users/gentemann/Google Drive/f_drive/secret_keys/tobias_login_eurec4a_v2.txt"
saildrone_key = get_key(file_key)

MQTT_USER = saildrone_key['username']
MQTT_PASSWORD = saildrone_key['password']

def _main():
    import paho.mqtt.client as mqtt
    client = mqtt.Client()
#    client.tls_set(ca_certs=os.path.join(os.path.dirname(__file__), "trustid-x3-root.pem.txt"))
    client.tls_set(ca_certs=os.path.join('C:/Users/gentemann/Google Drive/f_drive/secret_keys/', "trustid-x3-root.pem.txt"))
    client.username_pw_set(MQTT_USER, MQTT_PASSWORD)
    def on_connect(client, userdata, flags, rc):
        print("MQTT: connected with result code {}".format(rc))
        client.subscribe("platform/+/location")
    def on_disconnect(client, userdata, rc):
        print("MQTT: disonnected with result code {}".format(rc))
    def on_message(client, userdata, msg):
        topic = msg.topic.split("/")
        if topic[0] != "platform" or topic[2] != "location":
            return
        short_name = topic[1]
        print("{}: {}".format(short_name, msg.payload))
    def on_log(client, userdata, level, buf):
        print("#LOG: {} {}".format(level, buf))
    client.on_connect = on_connect
    client.on_disconnect = on_disconnect
    client.on_message = on_message
    client.on_log = on_log
    print("initial connect: {}".format(client.connect("mqtt.eurec4a.eu", 8883, 60)))
    client.loop_forever()
if __name__ == "__main__":
    _main()